In [1]:
require 'nyaplot'

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

true

In [2]:
require 'daru'

true

In [3]:
N = 10
M = [0, 1, 3, 9]

def normal_rand(mu = 0, sigma = 1.0)
  random = Random.new
  (Math.sqrt(-2 * Math.log(random.rand)) * Math.sin(2 * Math::PI * random.rand) * sigma) + mu
end


def create_dataset(num)
  dataset = Daru::DataFrame.new({'x': [], 'y': []})  

  num.times do |i|
    x = i.to_f / (num - 1).to_f
    y = Math.sin(2 * Math::PI * x) + normal_rand(0, 0.3)
    dataset.add_row(Daru::Vector.new([x, y], index: [:x, :y]))
  end
  
  return dataset
end

def rms_error(dataset, f)
  err = 0.0
  dataset.each_row do |line|
    err += 0.5 * (line.y - f.call(line.x))**2
  end

  Math.sqrt(2 * err / dataset.size)
end

def resolve(dataset, m)
  t = dataset.y

  columns = {}
  (m+1).times do |i|
    columns["x**#{i}"] = dataset.x ** i
  end
  phi = Daru::DataFrame.new(columns)

  tmp = (phi.transpose.to_matrix * phi.to_matrix).inv
  ws = (tmp * phi.transpose.to_matrix) * Vector.elements(t.to_a)

  f = lambda {|x|
    y = 0
    ws.each_with_index do |w, i|
      y += w * (x ** i)
    end

    y
  }

  return f, ws
end


:resolve

In [4]:
train_set = create_dataset(N)
test_set = create_dataset(N)
df_ws = {}

fig = Nyaplot::Frame.new

M.each_with_index do |m, c|
  f, ws = resolve(train_set, m)
  df_ws["M=#{m}"] = Daru::Vector.new(ws, name: "M=#{m}")
  
  plot = Nyaplot::Plot.new
  sc = plot.add_with_df(train_set.to_nyaplotdf, :scatter, :x, :y)
  sc.title("train_set")
  sc.color('blue')
  
  linex = (0..1).step(1.0 / (101 - 1)).to_a
  liney = linex.map do |x|
    Math.sin(2 * Math::PI * x)
  end
  line_answer = plot.add(:line, linex, liney)
  line_answer.title('answer')
  line_answer.color('green')
  
  liney = linex.map do |x|
    f.call(x)
  end
  line_erms = plot.add(:line, linex, liney)
  line_erms.title("E(RMS#{sprintf("%.2f", rms_error(train_set, f))})")
  line_erms.color('red')

  plot.configure do
    x_label("M=#{m}")
    y_label('')
    xrange([-0.05, 1.05])
    yrange([-1.5, 1.5])
    legend(true)
    height(300)
    width(490)
  end
  
  fig.add(plot)
end

puts 'Table ot the coefficients'
puts Daru::DataFrame.new(df_ws).inspect

fig.show

df = Daru::DataFrame.new({m: [], 'Training set': [], 'Test set': []})  
10.times do |m|
  f, ws = resolve(train_set, m)
  train_error = rms_error(train_set, f)
  test_error = rms_error(test_set, f)
  df.add_row(Daru::Vector.new([m, train_error, test_error], index: [:m, 'Training set'.to_sym, 'Test set'.to_sym]))
end

df.plot(type: [:line, :line], x: [:m, :m], y: ['Training set'.to_sym, 'Test set'.to_sym]) do |plot, diagrams|
  train_set_diagram = diagrams[0]
  test_set_diagram = diagrams[1]
  
  train_set_diagram.title('Training set')
  train_set_diagram.color('blue')
  test_set_diagram.title('Test set')
  test_set_diagram.color('green')
  
  plot.x_label('M')
  plot.y_label('RMS Error')
  plot.yrange([0, 0.9])
  plot.legend(true)
end

Table ot the coefficients
#<Daru::DataFrame(10x4)>
                   M=0        M=1        M=3        M=9
          0 -0.0651390 0.71921545 0.27349850 0.49988837
          1        nil -1.5687089 8.12152893 114.506346
          2        nil        nil -28.212160 -2833.6958
          3        nil        nil 19.8975562 26750.7509
          4        nil        nil        nil -128370.52
          5        nil        nil        nil 351243.492
          6        nil        nil        nil -572297.82
          7        nil        nil        nil 549787.641
          8        nil        nil        nil -287517.24
          9        nil        nil        nil 63122.5835


#<Nyaplot::Frame:0x007fce16791940 @properties={:panes=>[#<Nyaplot::Plot:0x007fce1676a430 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007fce1674a6a8 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :title=>"train_set", :color=>"blue"}, :data=>"ae32b05c-f9a1-423e-8362-a22561b7a81d"}, @xrange=[0.0, 1.0], @yrange=[-1.1169574555698307, 1.0334576976960939]>, #<Nyaplot::Diagram:0x007fce1623d390 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"answer", :color=>"green"}, :data=>"d44429e1-7560-4909-939f-7f77414fc3b0"}, @xrange=[0.0, 1.0], @yrange=[-1.0, 1.0]>, #<Nyaplot::Diagram:0x007fce161fb238 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"E(RMS0.75)", :color=>"red"}, :data=>"6c47e08e-186b-4eef-bc09-ca71c312082a"}, @xrange=[0.0, 1.0], @yrange=[-0.06513904563973032, -0.06513904563973032]>], :options=>{:x_label=>"M=0", :y_label=>"", :xrange=>[-0.05, 1.05], :yrange=>[-1.5, 1.5], :legend=>true, :height=>300, :width=>490, :zoom=>true}}>, #<Nyaplot::Plot:0x007fce1626db80 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007fce16286b08 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :title=>"train_set", :color=>"blue"}, :data=>"c0c5a9ef-bb06-4da9-9599-4683fae95bfc"}, @xrange=[0.0, 1.0], @yrange=[-1.1169574555698307, 1.0334576976960939]>, #<Nyaplot::Diagram:0x007fce16236068 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"answer", :color=>"green"}, :data=>"896b3846-edd0-4c9b-91d0-1be5715674aa"}, @xrange=[0.0, 1.0], @yrange=[-1.0, 1.0]>, #<Nyaplot::Diagram:0x007fce161e4f88 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"E(RMS0.56)", :color=>"red"}, :data=>"8b706b18-f24b-41d9-8530-128489016f26"}, @xrange=[0.0, 1.0], @yrange=[-0.8494935430485925, 0.7192154517691319]>], :options=>{:x_label=>"M=1", :y_label=>"", :xrange=>[-0.05, 1.05], :yrange=>[-1.5, 1.5], :legend=>true, :height=>300, :width=>490, :zoom=>true}}>, #<Nyaplot::Plot:0x007fce16025490 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007fce15fff420 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :title=>"train_set", :color=>"blue"}, :data=>"886033fb-cae3-462a-91dd-67165f77bf2e"}, @xrange=[0.0, 1.0], @yrange=[-1.1169574555698307, 1.0334576976960939]>, #<Nyaplot::Diagram:0x007fce15f77f48 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"answer", :color=>"green"}, :data=>"cd19e445-ac39-48a3-967a-09af2a81a8c4"}, @xrange=[0.0, 1.0], @yrange=[-1.0, 1.0]>, #<Nyaplot::Diagram:0x007fce15ebf2e0 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"E(RMS0.24)", :color=>"red"}, :data=>"60207172-e107-4590-96f6-ab6d37ebc8c4"}, @xrange=[0.0, 1.0], @yrange=[-1.116023227741172, 0.9373422535191966]>], :options=>{:x_label=>"M=3", :y_label=>"", :xrange=>[-0.05, 1.05], :yrange=>[-1.5, 1.5], :legend=>true, :height=>300, :width=>490, :zoom=>true}}>, #<Nyaplot::Plot:0x007fce15d09ea0 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007fce168bd940 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :title=>"train_set", :color=>"blue"}, :data=>"cd863231-72f7-48fd-b56f-d44c7a8f7deb"}, @xrange=[0.0, 1.0], @yrange=[-1.1169574555698307, 1.0334576976960939]>, #<Nyaplot::Diagram:0x007fce168467f0 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"answer", :color=>"green"}, :data=>"f0f9178c-d8dc-4860-a593-79b29623a726"}, @xrange=[0.0, 1.0], @yrange=[-1.0, 1.0]>, #<Nyaplot::Diagram:0x007fce16b4c7b0 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"E(RMS0.00)", :color=>"red"}, :data=>"42f826e9-56a9-4dfa-960d-536f68d0f70c"}, @xrange=[0.0, 1.0], @yrange=[-1.9180404627331882, 2.0111725232721365]>], :options=>{:x_label=>"M=9", :y_label=>"", :xrange=>[-0.05, 1.05], :yrange=>[-1.5, 1.5], :legend=>true, :height=>300, :width=>490, :zoom=>true}}>], :data=>{"ae32b05c-f9a1-423e-8362-a22561b7a81d"=>#<Nyaplot::DataFrame:0x007fce1674afb8 @name="ae32b05c-f9a1-423e-8362-a22561b7a81d", @rows=[{:x=>0.0, :y=>0.49989040133295687}, {:x=>0

#<Nyaplot::Frame:0x007fce16abf478 @properties={:panes=>[#<Nyaplot::Plot:0x007fce16af6c48 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007fce16ac44a0 @properties={:type=>:line, :options=>{:x=>:m, :y=>:"Training set", :title=>"Training set", :color=>"blue"}, :data=>"c4713cff-c1d4-4612-b491-01323b4c6f4d"}, @xrange=[0, 9], @yrange=[0.00011288602808587366, 0.7477820680605471]>, #<Nyaplot::Diagram:0x007fce16abfd88 @properties={:type=>:line, :options=>{:x=>:m, :y=>:"Test set", :title=>"Test set", :color=>"green"}, :data=>"c4713cff-c1d4-4612-b491-01323b4c6f4d"}, @xrange=[0, 9], @yrange=[0.410798637308882, 0.7901802535073036]>], :options=>{:x_label=>"M", :y_label=>"RMS Error", :yrange=>[0, 0.9], :legend=>true, :zoom=>true, :width=>800, :xrange=>[0, 9]}}>], :data=>{"c4713cff-c1d4-4612-b491-01323b4c6f4d"=>#<Nyaplot::DataFrame:0x007fce16ac4c98 @name="c4713cff-c1d4-4612-b491-01323b4c6f4d", @rows=[{:"Test set"=>0.7901802535073036, :"Training set"=>0.7477820680605471, :m=>0}, {:"Test set"=>0.6200049263932269, :"Training set"=>0.5554602636648857, :m=>1}, {:"Test set"=>0.6619201379663371, :"Training set"=>0.5357657333954127, :m=>2}, {:"Test set"=>0.4212656591284058, :"Training set"=>0.23861181391981123, :m=>3}, {:"Test set"=>0.4312430650725292, :"Training set"=>0.217134263059091, :m=>4}, {:"Test set"=>0.410798637308882, :"Training set"=>0.2035366396183082, :m=>5}, {:"Test set"=>0.4856996161055811, :"Training set"=>0.11275704213088493, :m=>6}, {:"Test set"=>0.49628366279645847, :"Training set"=>0.09302530588062327, :m=>7}, {:"Test set"=>0.5172729982325514, :"Training set"=>0.08474052811982946, :m=>8}, {:"Test set"=>0.5351527358812507, :"Training set"=>0.00011288602808587366, :m=>9}]>}, :extension=>[]}>